Comando abaixo apenas para que seja possível importar os modulos corretamente

In [1]:
import sys
sys.path.append("../../") 

# Criação de Features de Estilo

Nessa seção explicaremos como implementar as diferentes features de estilo.

## Como criar uma CharBasedFeature

Uma CharBasedFeature é uma classe de features textuais que calcula o resultado das métricas utilizando os caracteres do texto como parâmetro. Para criar uma feature charbased, fazemos:

###### Importamos os módulos necessários:

In [2]:
from feature.features import CharBasedFeature

###### Criamos a classe responsável pela feature:

In [3]:
class VogalsCount(CharBasedFeature):
    
    '''classe que contabiliza quantas vogais o texto possui'''
    
    def __init__(self,name,description,reference,visibility,text_format,feature_time_per_document,setChar=None,case_sensitive=False,ignore_punctuation=False):
        super(CharBasedFeature,self).__init__(name,description,reference,visibility,text_format,feature_time_per_document)
        
        if(setChar==None):
            setChar = []
        if(not case_sensitive):
            setChar = [letter.lower() for letter in setChar]
        
        self.case_sensitive = case_sensitive
        self.setChar = set(setChar)
        self.int_char_counter = 0
        self.ignore_punctuation = ignore_punctuation

A classe de exemplo acima herda de WordBasedFeature. Seus atributos são:

* setChar: conjunto de caracteres que serão contabilizados;
* case_sensitive: booleano que confirma se maiúsculas e minúsculas devem ser consideradas;
* ignore_punctuation: booleano que confirma se as pontuações do texto serão consideradas ou não.

###### Implementamos os métodos abstratos

Observemos a classe CharBasedFeature. A classe contém métodos abstratos que devem ser implementados nas classes filhas.

In [4]:
from feature.features import FeatureCalculator
from abc import abstractmethod

class CharBasedFeature(FeatureCalculator):

    @abstractmethod
    def checkChar(self,document,char):
        raise NotImplementedError

Como observado, a classe CharBasedFeature herda de FeatureCalculator. A classe FeatureCalculator também possui métodos abstratos que devem ser implementados. Abaixo o diagrama de classes que representa a estrutura de hierarquia.

![title](cbf.jpeg)

Em resumo, os métodos a serem implementados são checkChar, compute_feature e finish_document. O método checkChar é responsável por verificar cada caracter do texto. O método compute_feature é responsável por calcular o resultado da feature desejada, e o finish_document responsável por resetar as variáveis utilizadas no cálculo da feature.

Vamos implementar esses métodos na subclasse CharBasedFeature de exemplo.

In [5]:
class VogalsCount(CharBasedFeature):
    
    '''classe que contabiliza quantas vogais o texto possui'''
    
    def __init__(self,name,description,reference,visibility,text_format,feature_time_per_document,setChar=None,case_sensitive=False,ignore_punctuation=False):
        super(CharBasedFeature,self).__init__(name,description,reference,visibility,text_format,feature_time_per_document)
        
        if(setChar==None):
            setChar = []
        if(not case_sensitive):
            setChar = [letter.lower() for letter in setChar]
        
        self.case_sensitive = case_sensitive
        self.setChar = set(setChar)
        self.int_char_counter = 0
        self.ignore_punctuation = ignore_punctuation
        
    def checkChar(self, document, char):
        if self.ignore_punctuation is True and char in FeatureCalculator.word_divisors:
            return True
        if len(self.setChar) == 0 or char in self.setChar or (not self.case_sensitive and char.lower() in self.setChar):
            self.int_char_counter = self.int_char_counter + 1
        return True
    
    def compute_feature(self,document):
        return self.int_char_counter
    
    def finish_document(self,document):
        self.int_char_counter = 0

Seguindo esses passos, é possível implementar uma feature CharBasedFeature.

### Testando a feature implementada

Para fazer o teste manual da classe implementada, executamos seus métodos nas suas respectivas funções. Abaixo um código de teste para a execução da feature, em teste unitário.

In [6]:
import unittest
from feature.features import FeatureVisibilityEnum, Document, FeatureCalculator
from utils.basic_entities import FormatEnum, FeatureTimePerDocumentEnum

class TestFeatureCalculator(unittest.TestCase):
    
    def testCharCountTest(self):

            ccount = VogalsCount("VogalsCount", "Conta o número de vogais do texto", 
                                             "Jupyter Notebook documentation", 
                                             FeatureVisibilityEnum.public, 
                                             FormatEnum.text_plain, 
                                             FeatureTimePerDocumentEnum.MILLISECONDS,["a","e","i","o","u"],
                                             case_sensitive=True)

            document = Document(1,"doc1","O texto nao precisa -necessariamente - ser o texto que sera testado")
            
            for char in document.str_text:
                ccount.checkChar(document, char)
            
            int_result = ccount.compute_feature(document)
            print("O texto tem " + str(int_result) + " vogais.")
            ccount.finish_document(document)
            self.assertEqual(int_result, 25, "Nao foi contabilizado o numero de vogais correto no teste do primeiro documento")
    
sys.argv = ['', 'TestFeatureCalculator.testCharCountTest']
unittest.main()

.

O texto tem 25 vogais.



----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


SystemExit: False

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


O resultado do compute_feature deve ser 26, assim como o teste unitário.